## Data Characteristics & Challenge
**Type:** The dataset is **Tabular Data** representing clinical records.
**Challenge:** High missingness (~90% in some features).
**Solution:** We filtered columns with >90% missing values and rows with no data. For remaining missing values, we used **Median Imputation** because clinical variables (like CRP) often have skewed distributions (outliers), making the mean unreliable.

## Feature Scaling (Z-Score)
**Goal:** Prepare data for Clustering.
**Reasoning:** K-Means calculates **Euclidean Distances**. Without scaling, features with large values (e.g., Platelets ~200,000) would dominate features with small values (e.g., Creatinine ~1.0).
**Action:** Applied `StandardScaler` to transform all features to mean=0 and std=1.

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from pathlib import Path

# Paths
RAW_PATH = Path("../data/raw/dataset.xlsx")
OUT_PATH = Path("../data/processed/01_cleaned_data.csv")
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# Load
raw = pd.read_excel(RAW_PATH)
# Normalize column names
raw.columns = (raw.columns
               .str.replace(u"\xa0", " ", regex=False)
               .str.replace(r"\s+", " ", regex=True)
               .str.strip())

if "Patient ID" not in raw.columns:
    raise KeyError("'Patient ID' column is required.")

# Convert non-ID to numeric
non_id_cols = [c for c in raw.columns if c != "Patient ID"]
raw[non_id_cols] = raw[non_id_cols].apply(pd.to_numeric, errors="coerce")

# Deduplicate by patient ID
df = raw.drop_duplicates(subset=["Patient ID"]).copy()
feature_cols = [c for c in df.columns if c != "Patient ID"]

# Patient filter: >=10 non-missing labs
patient_counts = df[feature_cols].notna().sum(axis=1)
df = df.loc[patient_counts >= 10].copy()

# Feature filter: <80% missing
missing_frac = df[feature_cols].isna().mean()
kept_features = missing_frac[missing_frac < 0.80].index.tolist()
final_cols = ["Patient ID"] + kept_features

df = df[final_cols].copy()

# Impute + scale numeric features
numeric_cols = [c for c in df.columns if c != "Patient ID"]
imputer = SimpleImputer(strategy="median")
scaler = StandardScaler()

imputed = imputer.fit_transform(df[numeric_cols])
scaled = scaler.fit_transform(imputed)

processed = df.copy()
processed[numeric_cols] = scaled

print(f"Patients retained: {processed.shape[0]}")
print(f"Features retained: {len(numeric_cols)}")
print("Data were standardized (z-score) prior to saving.")

processed.to_csv(OUT_PATH, index=False)
print(f"Saved cleaned data to {OUT_PATH}")



Patients retained: 603
Features retained: 40
Data were standardized (z-score) prior to saving.
Saved cleaned data to ..\data\processed\01_cleaned_data.csv


### Data funnel and rationale (Quality over Quantity)
- **Funnel:** Initial patients → deduplicated → retained with ≤90% missing → final saved. See printed counts above for the exact numbers in this run.
- **Why discard ~90% missing profiles?** Patients with extremely sparse labs lack clinical signal (risk of garbage-in/garbage-out). We keep only those with a rich clinical profile (blood counts + metabolites) to ensure valid downstream inference.
- **Imputation is minimal:** Median imputation applied only after filtering to the retained patients/features; no broad filling of empty records.
- **Standardization:** Features are z-score scaled (StandardScaler) before export to align ranges for PCA/KMeans.
- **Handoff:** Cleaned, scaled dataframe is saved to `../data/processed/01_cleaned_data.csv` for the next notebook.

